In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sb

In [9]:
def get_stock(period, stock_ticker):
    stock = yf.Ticker(stock_ticker)
    #periods = “1d”, “5d”, “1mo”, “3mo”, “6mo”, “1y”, “2y”, “5y”, “10y”, “ytd”, “max”
    if period == '1d':
        interval = '1m'
    else:
        interval = '1d'
    return stock.history(period=period, interval=interval, actions=False)

In [10]:
amzn_df = get_stock(period='1mo', stock_ticker='amzn')

In [12]:
amzn_df.tail(10)

,Open,High,Low,Close,Volume
Date,,,,,
2021-10-15,3311.419922,3410.419922,3304.000000,3409.020020,5175100
2021-10-18,3388.360107,3449.169922,3385.100098,3446.739990,3174100
2021-10-19,3434.290039,3454.689941,3422.000000,3444.149902,2386100
2021-10-20,3452.659912,3462.860107,3400.370117,3415.060059,2139800
2021-10-21,3414.250000,3440.280029,3403.000000,3435.010010,1881400
2021-10-22,3421.000000,3429.840088,3331.300049,3335.550049,3133800
2021-10-25,3335.000000,3347.800049,3297.699951,3320.370117,2226000
2021-10-26,3349.510010,3416.120117,3343.979980,3376.070068,2698300
2021-10-27,3388.000000,3437.000000,3371.449951,3392.489990,2702200


In [14]:
amzn_df.mean()

Open      3.328810e+03
High      3.360402e+03
Low       3.302161e+03
Close     3.328730e+03
Volume    2.841755e+06
dtype: float64